In [62]:
import json

# Step 1: Load your JSON data
with open('/content/drive/MyDrive/SSMT/test_tokens_punct_labels_2.json', 'r') as file:
    dataset = json.load(file)

print(dataset)

[{'id': '12741024238657315067.wav', '1st_column': '1980', 'original_text': 'Unfortunately, studying traffic flow is difficult because driver behavior cannot be predicted with one-hundred percent certainty.', 'remove_punctuations': 'Unfortunately studying traffic flow is difficult because driver behavior cannot be predicted with onehundred percent certainty', 'tokens_per_sentence': ['en_XX', '▁Unfortunately', ',', '▁study', 'ing', '▁traffic', '▁flow', '▁is', '▁difficult', '▁because', '▁driver', '▁behavior', '▁cannot', '▁be', '▁predict', 'ed', '▁with', '▁one', '-', 'hund', 'red', '▁percent', '▁certain', 'ty', '.', '</s>'], 'total_tokens_per_sentence': 26, 'tokens_per_words': [[['▁Unfortunately', ','], 1], [['▁study', 'ing'], 6], [['▁traffic'], 6], [['▁flow'], 6], [['▁is'], 6], [['▁difficult'], 6], [['▁because'], 6], [['▁driver'], 6], [['▁behavior'], 6], [['▁cannot'], 6], [['▁be'], 6], [['▁predict', 'ed'], 6], [['▁with'], 6], [['▁one', '-', 'hund', 'red'], 5], [['▁percent'], 6], [['▁certa

In [63]:
def prepare_punctuation_data(dataset):
    prepared_data = []

    for item in dataset:
        original_text = item['original_text']
        no_punctuation_text = item['remove_punctuations']
        tokens_per_word = item['tokens_per_words']
        total_tokens = item["total_tokens_per_sentence"]
        tokens_per_sentence = item['tokens_per_sentence']

        # Prepare labels and tokens per word
        labels = []
        tokens_per_word_clean = []
        num_tokens_per_word = []

        # Iterate over each word and its tokens
        for word_info in tokens_per_word:
            token_sublist = word_info[0]  # List of tokens in the word
            # print(token_sublist)
            label = word_info[1]  # Label for the word
            # print(label)

            # Repeat the label for each token in the word
            labels.extend([label] * len(token_sublist))
            num_tokens_per_word.append(len(token_sublist))
            # print('final labels',labels)

            # Add the tokens themselves (no extra nesting)
            tokens_per_word_clean.extend(token_sublist)

        # Collect the information into a dictionary for the current item
        prepared_data.append({
            'id': item['id'],  # ID from previous json
            'original_text': original_text,  # Original text from previous json
            'no_punctuation_text': no_punctuation_text,  # Text without punctuation
            'tokens_per_word': tokens_per_word_clean,  # Cleaned tokens per word (no extra nesting)
            'total_tokens': total_tokens,  # Total number of tokens from previous json
            'tokens_per_sentence': tokens_per_sentence,  # Tokens per sentence from previous json
            'labels': labels,  # Labels for each token,
            'num_tokens_per_word': num_tokens_per_word  # Number of tokens per word
        })

    return prepared_data


In [64]:
import json

final_data = prepare_punctuation_data(dataset)

with open('test_punctuation_data_speech.jsonl', 'w') as f:
    for item in final_data:
        json.dump(item, f)
        f.write('\n')


In [65]:
import json

# Open the jsonl file and read the labels
tokens = []

with open('test_punctuation_data_speech.jsonl', 'r') as f:
    for line in f:
        # Load each line as a JSON object
        item = json.loads(line)

        # Extract the 'labels' key
        tokens.append(item)

print(tokens[0])

{'id': '12741024238657315067.wav', 'original_text': 'Unfortunately, studying traffic flow is difficult because driver behavior cannot be predicted with one-hundred percent certainty.', 'no_punctuation_text': 'Unfortunately studying traffic flow is difficult because driver behavior cannot be predicted with onehundred percent certainty', 'tokens_per_word': ['▁Unfortunately', ',', '▁study', 'ing', '▁traffic', '▁flow', '▁is', '▁difficult', '▁because', '▁driver', '▁behavior', '▁cannot', '▁be', '▁predict', 'ed', '▁with', '▁one', '-', 'hund', 'red', '▁percent', '▁certain', 'ty', '.'], 'total_tokens': 26, 'tokens_per_sentence': ['en_XX', '▁Unfortunately', ',', '▁study', 'ing', '▁traffic', '▁flow', '▁is', '▁difficult', '▁because', '▁driver', '▁behavior', '▁cannot', '▁be', '▁predict', 'ed', '▁with', '▁one', '-', 'hund', 'red', '▁percent', '▁certain', 'ty', '.', '</s>'], 'labels': [1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 6, 2, 2, 2], 'num_tokens_per_word': [2, 2, 1, 1, 1, 1, 1

In [66]:
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('/content/drive/MyDrive/SSMT/test_word_level_features.mat')

# Print the keys of the dictionary to understand its structure
print(mat_data.keys())

features = mat_data['word_features'].flatten()
filenames = mat_data['filenames'].flatten().tolist()
filenames = [f.strip() for f in filenames]

words = mat_data['words'].flatten().tolist()

# cleaned_array = np.char.strip(your_array)

print("Features shape:", features.shape)

# Now you can use the features array as needed


dict_keys(['__header__', '__version__', '__globals__', 'word_features', 'words', 'filenames'])
Features shape: (556,)


In [67]:
failed = 0
final_features = []
final_filenames = []
tokenss = []
ff = []
labelss = []
for i in range(0,len(filenames)):
  filenames[i] = filenames[i] + '.wav'
  # print(filenames[i])
  # print(features[i].shape)
  # print(words[i])
  # print(filenames[i])
  t = []
  for f in range(0,len(tokens)):
    if tokens[f]['id'] == filenames[i]:
      # print(tokens[f]['id'])
      # print(f)
      id = tokens[f]['id']
      original_text = tokens[f]['original_text']
      no_punctuation_text = tokens[f]['no_punctuation_text']
      # print(len(no_punctuation_text.split()))
      tokens_per_word = tokens[f]['tokens_per_word']
      total_tokens = tokens[f]['total_tokens']
      tokens_per_sentence = tokens[f]['tokens_per_sentence']
      labels = tokens[f]['labels']
      num_tokens_per_word = tokens[f]['num_tokens_per_word']
      if features[i].shape[0] != len(no_punctuation_text.split()):
        # continue
        # print(features[i].shape[0], '--------------', len(no_punctuation_text.split()))
        failed +=1
        break
      # for i in num_tokens_per_word:
      #   t.append(i)
      ff.append(id)
      tokenss.append(num_tokens_per_word)
      labelss.append(labels)


  # break
print(failed)
print(tokenss)

156
[[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 3], [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 3], [4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 3], [4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 3], [1, 3, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3], [1, 3, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3], [1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 3], [1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 3], [3, 1, 1, 1, 2, 1, 1, 1, 1, 5, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 3, 2, 1, 1, 3], [3, 1, 1, 1, 2, 1, 1, 1, 1, 5, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 3, 2, 1, 1, 3], [1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2], [1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2], [1, 4, 1, 1, 3, 1, 1, 3, 1, 1, 3], [1, 4, 1, 1, 3, 1, 1, 3, 1, 1, 3], [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2], [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2], [1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2], [1, 1, 1, 2, 

In [68]:
print(len(tokenss))
print(len(ff))
print(len(labelss))
print(tokenss[0])
print(ff[0])

400
400
400
[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 3]
12741024238657315067.wav


In [69]:
import numpy as np
import scipy.io

all_repeated_features = []
all_repeated_filenames = []
all_labels = []
not_found_count = 0
mismatch_count = 0

for i in range(len(filenames)):
    mat_filename = filenames[i].strip()

    if mat_filename in ff:
        idx = ff.index(mat_filename)
        token_counts = tokenss[idx]
        l = labelss[idx]
        feature_matrix = features[i]

        if len(token_counts) != feature_matrix.shape[0]:
            mismatch_count += 1
            continue

        repeated_features = []
        for row, repeat_count in zip(feature_matrix, token_counts):
            repeated_features.extend([row] * repeat_count)

        repeated_features = np.array(repeated_features)
        all_repeated_features.append(repeated_features)
        all_repeated_filenames.append(mat_filename)
        all_labels.append(l)
    else:
        not_found_count += 1

print("Files not found in ff:", not_found_count)
print("Files with length mismatch:", mismatch_count)
print("Total files processed:", len(all_repeated_features))

# Save to .mat file
scipy.io.savemat('/content/drive/MyDrive/SSMT/test_speech_subtoken_punc_features.mat', {
    'features': all_repeated_features,
    'filenames': all_repeated_filenames,
    'labels': all_labels
})


Files not found in ff: 156
Files with length mismatch: 1
Total files processed: 399
